In [ ]:
#Importing necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay # Corrected the import statement to use ConfusionMatrixDisplay instead of ConfusionMatrixDisplay3

In [ ]:
#Loading data
bank = pd.read_csv("/content/bank (1).csv",sep=';')


In [ ]:
bank.head()

In [ ]:
bank.shape

In [ ]:
bank.info()

In [ ]:
#Statistical Analysis
bank.describe()

In [ ]:
#Checking Missing values
bank.isnull().sum()

In [ ]:
#Checking for duplicates
bank.duplicated().sum()

In [ ]:
#Investigating these 12 duplicates
bank[bank.duplicated()]

In [ ]:
sns.histplot(x="age", data=bank, kde=True, hue= "y")
plt.title("Age Distribution and Deposits\n")
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
sns.countplot(x="job", data= bank, hue ="y")
plt.title("Occupation Distribution and Deposits\n")
plt.show()

In [ ]:
plt.figure(figsize=(7,3))
sns.countplot(x="marital", data= bank, hue ="y")
plt.title("Marital Status and Deposits\n")
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(x="education", data= bank, hue ="y")
plt.title("Education Status and Deposits\n")
plt.show()

In [ ]:
bank.default.value_counts()

In [ ]:
plt.figure(figsize=(6,3.5))
sns.countplot(x="housing", data= bank, hue ="y")
plt.title("Housing Loan Distribution and Deposits\n")
plt.show()

In [ ]:
plt.figure(figsize=(6,3.5))
sns.countplot(x="loan", data= bank, hue ="y")
plt.title("Personal Loan Distribution and Deposits\n")
plt.show()

In [ ]:
#Count of Outcome
bank.y.value_counts()
keys = bank.y.value_counts().index
data = bank.y.value_counts().values
plt.figure(figsize=(6,3.5))
explode = [0,0.1]
plt.pie(data,labels=keys,explode=explode, autopct='%.0f%%')
plt.show()

In [ ]:
cols = bank.select_dtypes("object").columns
cols

In [ ]:
le = LabelEncoder()

bank[cols] = bank[cols].apply(le.fit_transform)

In [ ]:
bank.head(3)

In [ ]:
plt.figure(figsize=(23,10))
sns.heatmap(bank.corr(), cmap='bwr', annot=True)
plt.show()

In [ ]:
#Splitting input and output
X = bank.drop("y", axis=1)
y = bank.y

In [ ]:
scaler = StandardScaler()

X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [ ]:
#Train-test split
train_X, test_X, train_y, test_y = train_test_split(X_scaled, y, test_size=0.3)

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_X, train_y)

In [ ]:
print('Train Score: {}'.format(decision_tree.score(train_X, train_y)))
print('Test Score: {}'.format(decision_tree.score(test_X, test_y)))

In [ ]:
cross_val_score(decision_tree, train_X, train_y, cv=5).mean()

In [ ]:
ypred = decision_tree.predict(test_X)
print(classification_report(test_y,ypred))

In [ ]:
#Applying Grid search cv to find best estimaters to improve model performance

param_grid = {
    'max_depth': [3, 5, 7,10, None],
    'criterion' : ['gini', 'entropy'],
    'min_samples_leaf': [3, 5, 7, 9,10,20]
    }

In [ ]:
gscv = GridSearchCV(decision_tree, param_grid, cv=5, verbose=1)
gscv.fit(train_X, train_y)

In [ ]:
gscv.best_params_

In [ ]:
gscv.best_estimator_

In [ ]:
cross_val_score(gscv.best_estimator_, train_X, train_y, cv=5).mean()

In [ ]:
clf = DecisionTreeClassifier(criterion= 'gini', max_depth= 5, min_samples_leaf = 3)
clf.fit(train_X, train_y)

In [ ]:
print('Train Score: {}'.format(clf.score(train_X, train_y)))
print('Test Score: {}'.format(clf.score(test_X, test_y)))

In [ ]:
pred_y = clf.predict(test_X)

In [ ]:
#Confusion Matrix
cm = confusion_matrix(pred_y, test_y)
ConfusionMatrixDisplay(cm, display_labels=clf.classes_).plot()
plt.show()

In [ ]:
#Classification Report
print(classification_report(pred_y, test_y))

In [ ]:
#Accuracy Score
accuracy = accuracy_score(test_y,pred_y)
print("Test Accuracy of Decision Tree Classifier : {}".format(accuracy*100))

In [ ]:
#Cross Validation Score
Cross_val = cross_val_score(clf, test_X,test_y, cv=5).mean()
print("Cross-Validation Accuracy Scores Decision Tree : ",Cross_val*100)

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(25,20))
t= tree.plot_tree(clf,filled=True,feature_names=X.columns)